In [ ]:
import numpy as np
import pandas as pd
from keras.layers import LSTM , Dense ,Dropout
from keras.models import Sequential

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
train_dcast = pd.crosstab(index=[train['PID']], columns=train['Event'])
train_dcast.reset_index(drop=False, inplace=True)

In [ ]:
train_dcast.head()

In [ ]:
submit = train_dcast.loc[:,train_dcast.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, 
                          index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
submit.drop('index',inplace=True, axis=1)

In [ ]:
# setting column names in sequence
submit['PID'] = train_dcast['PID']
cols = submit.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit = submit[cols]

In [ ]:
submit.head()


In [ ]:
x = train_dcast['PID']

In [ ]:
y = train_dcast.drop(['PID'],axis = 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()

x_train = sc.fit_transform(x)
x_train = np.reshape(x_train,(3000,1,1))

y_train = sc.fit_transform(y)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from keras.layers import Dense,LSTM
from keras.models import Sequential

In [ ]:
reg = Sequential()
reg.add(LSTM(units =4,activation = 'sigmoid',input_shape = (None , 1)))
reg.add(Dense(units = 6472,kernel_initializer = 'uniform'))
reg.add(Dense(units = 6472 , kernel_initializer = 'uniform'))
reg.add(Dense(units=6472,kernel_initializer='uniform'))
reg.compile(optimizer='adam',loss='mean_squared_error')
reg.fit(x_train,y_train,batch_size = 50,epochs = 200)
test = pd.read_csv('test.csv')